In [1]:
# Install dependencies
import sys
!{sys.executable} -m pip install numpy matplotlib pandas openpyxl sklearn seaborn matplotlib

  Using cached numpy-1.24.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached matplotlib-3.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 63.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 93.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 112.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 97.7 MB/s eta 0:00:00
  Using cached Pillow-9.5.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.4 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached contourpy-1.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34

In [10]:
# Imports and global options
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import multiprocessing
from functools import partial

In [11]:
# prepare dataset
def prepare_dataset(dataset_file, input_cols, result_col=None):
    # Load data
    df = pd.read_excel(dataset_file)
    
    # Filter bad data, replace categories with dummies
    x_df = df[input_cols].apply(lambda x: x.str.strip())
    x = pd.get_dummies(x_df, drop_first=True)
    
    if result_col:
        y_df = df[result_col].str.strip()
        y = pd.get_dummies(y_df, drop_first=True).iloc[:, 0]
        
        return x, y
    else:
        return x

In [12]:
# Data and dummies displayed in a table
def display_input_data(dataset_file, input_cols):
    x = prepare_dataset(dataset_file, input_cols)
    display(x)

In [13]:
def check_input_data_indices(train_dataset_file, test_dataset_file, input_cols):
    x_train = prepare_dataset(train_dataset_file, input_cols)
    x_test = prepare_dataset(test_dataset_file, input_cols)
    
    if not x_train.columns.equals(x_test.columns):
        raise Exception('Datasets are incompatible!');
    print('Datasets are compatible')

In [14]:
def train_model(train_dataset_file, input_cols, result_col):
    x, y = prepare_dataset(train_dataset_file, input_cols, result_col)
    
    # Instantiate the model
    model = LogisticRegression(solver='liblinear', random_state=0)

    # Train (=fit) the model
    model.fit(x, y)
    
    return model


In [15]:
def evaluate_model(model, test_dataset_file, input_cols, result_col):
    x, y = prepare_dataset(test_dataset_file, input_cols, result_col)
    
    return model.score(x, y)

In [16]:
def _evaluate_for_input_combination(train_dataset_file, test_dataset_file, input_combination, result_col):
    #print(input_cols_comb)
    model = train_model(
        train_dataset_file,
        list(input_combination),
        result_col
    )
    score = evaluate_model(
        model,
        test_dataset_file,
        list(input_combination),
        result_col
    )
    #print(score)
    return score

def run_input_optimizer(train_dataset_file, test_dataset_file, input_cols, result_col):
    # Contains all combinations by input length (3D list)
    input_combinations = []
    for l in range(2, len(input_cols) + 1):
        input_combinations.append(list(itertools.combinations(input_cols, l)))

    scores = []
    flat_input_combinations = []
    
    for input_combinations_of_len in input_combinations:
        print('Testing combinations of length ' + str(len(input_combinations_of_len[0])))
        
        for input_combination in input_combinations_of_len:
            score = _evaluate_for_input_combination(train_dataset_file, test_dataset_file, input_combination, result_col)
            scores.append(score)
            flat_input_combinations.append(input_combination)

    # Pick and print results
    max_score = max(scores)
    max_score_idx = scores.index(max_score)
    max_score_cols = flat_input_combinations[max_score_idx]

    min_score = min(scores)
    min_score_idx = scores.index(min_score)
    min_score_cols = flat_input_combinations[min_score_idx]

    print('Max score: ' + str(max_score))
    print('Best columns combination: ' + str(max_score_cols))
    print('Min score: ' + str(min_score))
    print('Worse columns combination: ' + str(min_score_cols))